# Sentiment Analysis

[1. Sentiment Analysis with Regular Classifier](#1)<br>
[2. Sentiment Analysis with RNN](#2)<br>

# <a id='0'>0: Introduction

Sentiment Analysis (SA) is one of the most popular uses of NLP these days.

SA has become an important tool for many purposes. Some examples are:
* Customer sentiment in a company for financial anaylsis and investment decisions
* Social Media feedback for advertising campaigns
* Quantitative measure for book and movie reviews

This section will be relatively short as the topic is simple once you know classification algorithms. It will have two labs where in each you will train a classifier on movie reviews from the Internet Movie Database (IMDB) dataset.

Labs:
1. Regular supervised learning classifier looking at a review as an unorganized collection of words
2. Exploit the sequentiality of the sentences and will train a more accurate classifier using a Recurrent Neural Network (RNN)

# <a id='1'>1: Sentiment Analysis with Regular Classifier</a>

# <a id='2'>2: Sentiment Analysis with RNN</a>